# Data preparation for Neural Network sales forecasting (Kaggle-style)

This notebook prepares `data_org.csv` for a neural network model.

**Goal (supervised learning):** predict **daily sales per Product Group** (target = `Sales Volume`) using:
- calendar features (day of week, month)
- special days (`is_holiday`, `KielerWoche`)
- weather features (Temperature, etc.)
- lag & rolling features per Product Group (leakage-safe)

**Kaggle split concept**
- **Train**: labels known → fit model
- **Validation**: labels known → compare models
- **Test**: labels unknown → create features and generate predictions later (this notebook exports test features too)

> Important: We do **not** use `daily_total_sales` as a feature because it leaks information from the target.


In [2]:
# Imports
import os
import numpy as np
import pandas as pd

# Path to your data
DATA_PATH = "data_prep/data_org.csv"   # adjust if your folder differs

# Load
data = pd.read_csv(DATA_PATH)

# Basic checks
print("Shape:", data.shape)
print("Columns:", list(data.columns))
data.head()


Shape: (11164, 17)
Columns: ['id', 'Date', 'Product Group', 'Sales Volume', 'Cloud Cover', 'Temperature', 'Wind Speed', 'Weather Code', 'Temperature_H', 'Wind Speed_H', 'Cloud Cover_H', 'Precipitation_H', 'KielerWoche', 'Holiday Name (English)', 'month', 'day_of_week', 'daily_total_sales']


,id,Date,Product Group,Sales Volume,Cloud Cover,Temperature,Wind Speed,Weather Code,Temperature_H,Wind Speed_H,Cloud Cover_H,Precipitation_H,KielerWoche,Holiday Name (English),month,day_of_week,daily_total_sales
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,15.6,3.5,0.6875,0.3,0,NaN,7,0,1269.249107
1,1307013,2013-07-01,3,201.198426,6.0,17.8375,15.0,20.0,15.6,3.5,0.6875,0.3,0,NaN,7,0,1269.249107
2,1307014,2013-07-01,4,65.890169,6.0,17.8375,15.0,20.0,15.6,3.5,0.6875,0.3,0,NaN,7,0,1269.249107
3,1307015,2013-07-01,5,317.475875,6.0,17.8375,15.0,20.0,15.6,3.5,0.6875,0.3,0,NaN,7,0,1269.249107
4,1307012,2013-07-01,2,535.856285,6.0,17.8375,15.0,20.0,15.6,3.5,0.6875,0.3,0,NaN,7,0,1269.249107


## Build clean features (date parsing, holidays, lags/rolling, temperature imputation)

We create the following:
- `day_of_week`, `month`, `doy` from `Date`
- `is_holiday` from `Holiday Name (English)` (1 if not null)
- temperature imputation for dates where Temperature is missing (common in Kaggle test) using **historical day-of-year mean**
- lag features (`lag_1`, `lag_7`, `lag_14`) and rolling features (`roll_mean_7`, `roll_mean_28`, `roll_std_28`) per Product Group

Lag/rolling features are **leakage-safe** because we compute them using `.shift(1)` before rolling.


In [3]:
# --- Parse date and create calendar features ---
data["Date"] = pd.to_datetime(data["Date"])

data["day_of_week"] = data["Date"].dt.dayofweek   # 0=Mon ... 6=Sun
data["month"] = data["Date"].dt.month
data["doy"] = data["Date"].dt.dayofyear

# --- Holiday flag ---
holiday_col = "Holiday Name (English)" if "Holiday Name (English)" in data.columns else None
if holiday_col:
    data["is_holiday"] = data[holiday_col].notna().astype(int)
else:
    data["is_holiday"] = 0

# KielerWoche should already exist; if not, create a default 0
if "KielerWoche" not in data.columns:
    data["KielerWoche"] = 0

# --- Temperature handling ---
# Kaggle reality: TEST often has no Temperature. We will impute Temperature using historical day-of-year mean.

if "Temperature" not in data.columns:
    data["Temperature"] = np.nan

data["Temperature"] = pd.to_numeric(data["Temperature"], errors="coerce")

# Compute DOY mean temperature using only rows where Temperature is known
doy_mean_temp = (
    data.loc[data["Temperature"].notna()]
        .groupby("doy")["Temperature"]
        .mean()
)

# Fill missing temps from DOY mean, then fallback to overall mean
data["Temperature"] = data["Temperature"].fillna(data["doy"].map(doy_mean_temp))
data["Temperature"] = data["Temperature"].fillna(data["Temperature"].mean())

# (Optional) Temperature anomaly: useful if you want to separate seasonal vs unexpected temperature effects
data["temp_anomaly"] = data["Temperature"] - data["doy"].map(doy_mean_temp)
data["temp_anomaly"] = data["temp_anomaly"].fillna(0.0)

# --- Build lag/rolling features per Product Group ---
# Target column
TARGET_COL = "Sales Volume"

# Ensure numeric
data[TARGET_COL] = pd.to_numeric(data[TARGET_COL], errors="coerce")

# Sort for time-series feature engineering
data = data.sort_values(["Product Group", "Date"]).reset_index(drop=True)

g = data.groupby("Product Group")[TARGET_COL]

data["lag_1"] = g.shift(1)
data["lag_7"] = g.shift(7)
data["lag_14"] = g.shift(14)

data["roll_mean_7"] = g.shift(1).rolling(window=7, min_periods=3).mean()
data["roll_mean_28"] = g.shift(1).rolling(window=28, min_periods=7).mean()
data["roll_std_28"] = g.shift(1).rolling(window=28, min_periods=7).std()

# Fill NaNs in lag/rolling features (early history) with 0 for modeling convenience
for c in ["lag_1","lag_7","lag_14","roll_mean_7","roll_mean_28","roll_std_28"]:
    data[c] = data[c].fillna(0.0)

print("Prepared shape:", data.shape)
data.head()


Prepared shape: (11164, 26)


,id,Date,Product Group,Sales Volume,Cloud Cover,Temperature,Wind Speed,Weather Code,Temperature_H,Wind Speed_H,...,daily_total_sales,doy,is_holiday,temp_anomaly,lag_1,lag_7,lag_14,roll_mean_7,roll_mean_28,roll_std_28
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,15.6,3.5,...,1269.249107,182,0,-1.282143,0.000000,0.0,0.0,0.000000,0.0,0.0
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,NaN,15.1,2.7,...,1430.008397,183,0,-2.100000,148.828353,0.0,0.0,0.000000,0.0,0.0
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,61.0,18.0,2.3,...,1124.274894,184,0,1.026786,159.793757,0.0,0.0,0.000000,0.0,0.0
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,20.0,17.2,2.2,...,1184.309567,185,0,-1.995833,111.885594,0.0,0.0,140.169235,0.0,0.0
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,NaN,16.9,3.7,...,1288.023060,186,0,0.075000,168.864941,0.0,0.0,147.343161,0.0,0.0


## Time-based Train / Validation / Test split

We keep your established split:
- Train: 2013-07-01 → 2017-07-31  
- Validation: 2017-08-01 → 2018-07-31  
- Test: 2018-08-01 → 2019-07-31

This respects time order and avoids leakage.


In [4]:
# Time splits
TRAIN_START, TRAIN_END = "2013-07-01", "2017-07-31"
VAL_START, VAL_END     = "2017-08-01", "2018-07-31"
TEST_START, TEST_END   = "2018-08-01", "2019-07-31"

train = data[(data["Date"] >= TRAIN_START) & (data["Date"] <= TRAIN_END)].copy()
val   = data[(data["Date"] >= VAL_START)   & (data["Date"] <= VAL_END)].copy()
test  = data[(data["Date"] >= TEST_START)  & (data["Date"] <= TEST_END)].copy()

print("Train:", train["Date"].min(), "→", train["Date"].max(), train.shape)
print("Val:  ", val["Date"].min(), "→", val["Date"].max(), val.shape)
print("Test: ", test["Date"].min(), "→", test["Date"].max(), test.shape)

# Kaggle test often has missing target labels (Sales Volume). That's fine.
print("Test target missing %:", test[TARGET_COL].isna().mean() * 100)


Train: 2013-07-01 00:00:00 → 2017-07-31 00:00:00 (7493, 26)
Val:   2017-08-01 00:00:00 → 2018-07-31 00:00:00 (1841, 26)
Test:  2018-08-01 00:00:00 → 2019-07-30 00:00:00 (1830, 26)
Test target missing %: 100.0


## Encode features for neural networks (one-hot categoricals, scale numeric)

We build:
- `X_train`, `y_train`
- `X_val`, `y_val`
- `X_test` (no labels needed)

**Categorical columns** are one-hot encoded.
**Numeric columns** are standardized (mean 0, std 1) using TRAIN statistics only.


In [5]:
from sklearn.preprocessing import StandardScaler

# Columns to exclude from features (leakage or identifiers)
exclude_cols = {
    TARGET_COL,          # target
    "Date",              # keep for indexing, not modeling
    "daily_total_sales", # leakage: sum of all product sales
    "Holiday Name (English)",  # text label (we already created is_holiday)
}

# Categorical columns (you can add/remove here)
categorical_cols = ["Product Group", "day_of_week", "month", "is_holiday", "KielerWoche"]

# If you want to let NN learn different weather sensitivity by product group, keep Product Group categorical.
# Note: day_of_week/month could also be encoded cyclically (sin/cos), but one-hot is a good baseline.

# Select feature columns automatically:
feature_cols = [c for c in data.columns if c not in exclude_cols]

# Make sure categorical columns exist (if dataset changes)
categorical_cols = [c for c in categorical_cols if c in feature_cols]

# Split X/y
def split_xy(df):
    X = df[feature_cols].copy()
    y = df[TARGET_COL].copy() if TARGET_COL in df.columns else None
    return X, y

X_train_raw, y_train = split_xy(train)
X_val_raw, y_val     = split_xy(val)
X_test_raw, _        = split_xy(test)

# One-hot encode categoricals (fit on combined to ensure consistent columns)
combined = pd.concat([X_train_raw, X_val_raw, X_test_raw], axis=0, ignore_index=True)
combined = pd.get_dummies(combined, columns=categorical_cols, drop_first=False)

# Re-split after encoding
n_tr = len(X_train_raw)
n_va = len(X_val_raw)

X_train_enc = combined.iloc[:n_tr].copy()
X_val_enc   = combined.iloc[n_tr:n_tr+n_va].copy()
X_test_enc  = combined.iloc[n_tr+n_va:].copy()

# Identify numeric columns to scale
numeric_cols = [c for c in X_train_enc.columns if X_train_enc[c].dtype != "uint8" and X_train_enc[c].dtype != "bool"]

# Scale numeric columns using TRAIN stats only
scaler = StandardScaler()
X_train_enc[numeric_cols] = scaler.fit_transform(X_train_enc[numeric_cols])
X_val_enc[numeric_cols]   = scaler.transform(X_val_enc[numeric_cols])
X_test_enc[numeric_cols]  = scaler.transform(X_test_enc[numeric_cols])

# Final outputs (numpy arrays are NN-friendly)
X_train = X_train_enc.to_numpy(dtype=np.float32)
X_val   = X_val_enc.to_numpy(dtype=np.float32)
X_test  = X_test_enc.to_numpy(dtype=np.float32)

y_train = y_train.to_numpy(dtype=np.float32)
y_val   = y_val.to_numpy(dtype=np.float32)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:  ", X_val.shape, "y_val:  ", y_val.shape)
print("X_test: ", X_test.shape)

# Keep feature names for later model interpretation/debugging
feature_names = list(X_train_enc.columns)
print("Num features:", len(feature_names))
feature_names[:20]


X_train: (7493, 46) y_train: (7493,)
X_val:   (1841, 46) y_val:   (1841,)
X_test:  (1830, 46)
Num features: 46


['id',
 'Cloud Cover',
 'Temperature',
 'Wind Speed',
 'Weather Code',
 'Temperature_H',
 'Wind Speed_H',
 'Cloud Cover_H',
 'Precipitation_H',
 'doy',
 'temp_anomaly',
 'lag_1',
 'lag_7',
 'lag_14',
 'roll_mean_7',
 'roll_mean_28',
 'roll_std_28',
 'Product Group_1',
 'Product Group_2',
 'Product Group_3']

## Export prepared arrays for your Neural Network notebook

We export:
- `X_train`, `y_train`
- `X_val`, `y_val`
- `X_test`
- `feature_names`
- fitted `scaler` (so you can apply the same transform later if needed)
- `test_ids` (so predictions can be mapped back to Kaggle submission)



In [6]:
import pickle

subdirectory = "pickle_data"
os.makedirs(subdirectory, exist_ok=True)

# IDs for Kaggle submission
test_ids = test["id"].copy() if "id" in test.columns else (
    test["Date"].dt.strftime("%y%m%d") + test["Product Group"].astype(str)
)

with open(os.path.join(subdirectory, "nn_data.pkl"), "wb") as f:
    pickle.dump(
        {
            "X_train": X_train,
            "y_train": y_train,
            "X_val": X_val,
            "y_val": y_val,
            "X_test": X_test,
            "test_ids": test_ids.to_numpy(),
            "feature_names": feature_names,
            "scaler": scaler,
        },
        f,
        protocol=pickle.HIGHEST_PROTOCOL,
    )

print("Saved:", os.path.join(subdirectory, "nn_data.pkl"))


Saved: pickle_data\nn_data.pkl
